In [ ]:
# (The NHL and the NHL Shield are registered trademarks of the 
# National Hockey League. NHL and NHL team marks are the property of 
# the NHL and its teams. © NHL 2023. All Rights Reserved.)

import pandas as pd
import datetime
import os
import requests
import json
import matplotlib.pyplot as plt
import numpy as np
import hvplot.pandas
import geopandas as gpd
import seaborn as sns
import scipy.stats as st
from sqlalchemy import create_engine
from random import randint
from time import sleep
from scipy.stats import pearsonr, mannwhitneyu



In [ ]:
# Get Stanely Cup Winning and Losing teams since 1927
# Code by Ron Brennan

url = "https://en.wikipedia.org/wiki/List_of_Stanley_Cup_champions"

stanleycup_champs = pd.read_html(url)

stanleycup_byyear_df = stanleycup_champs[2]
stanleycup_byyear_df.head()

In [ ]:
# Find Winning Teams from 1991-2023
# Code by Ron Brennan

stanleycup_winner_cleaned_df = stanleycup_byyear_df.loc[stanleycup_byyear_df['Year'].between(1992,2023)]
stanleycup_winner_cleaned_df = stanleycup_winner_cleaned_df[["Year", "Winning team"]]

stanleycup_winner_cleaned_df.reset_index(drop=True, inplace=True)

stanleycup_winner_cleaned_df["Year"] = (stanleycup_winner_cleaned_df["Year"] -1).map(str) + (stanleycup_winner_cleaned_df["Year"]).map(str)
stanleycup_winner_cleaned_df["Winning team"] = stanleycup_winner_cleaned_df["Winning team"].str.split('(').str[0]

stanleycup_winner_cleaned_df.drop(stanleycup_winner_cleaned_df[(stanleycup_winner_cleaned_df['Year'] == "20042005")].index, inplace=True)
stanleycup_winner_cleaned_df.rename(columns={"Winning team":"Team Name"}, inplace=True)
stanleycup_winner_cleaned_df["Team Name"] = stanleycup_winner_cleaned_df["Team Name"].str.strip()

# Change names 
stanleycup_winner_cleaned_df["Team Name"] = stanleycup_winner_cleaned_df["Team Name"].str.replace("Montreal Canadiens", "Montréal Canadiens")
stanleycup_winner_cleaned_df

In [ ]:
# Get Team Name and associated with its ID in the NHL API JSON request

teamID_url = "https://statsapi.web.nhl.com/api/v1/teams"
   
response = requests.get(teamID_url)

response_json = response.json()
team_info = {}

for team in response_json["teams"]:
    team_id = team["id"]
    team_name = team["name"]
    team_info[team_id] = team_name
   
team_info_df = pd.DataFrame.from_dict(team_info, orient="index", columns=["Team Name"])
team_info_df.reset_index(inplace=True)
team_info_df.columns = ["Team ID", "Team Name"]

team_info_df

In [ ]:
# Merge dataframes to associate the Winning Team and Season won with the Team ID

stanleycup_winning_merge_byname_ID = pd.DataFrame.merge(stanleycup_winner_cleaned_df,team_info_df, on='Team Name', how='left')
stanleycup_winning_merge_byname_ID

In [ ]:
# Get Roster for each Winning team per Season.  
# Will use this code

player_list = []
base_url = "https://statsapi.web.nhl.com/api/v1/teams/{}?expand=team.roster&season={}"
for index, row in stanleycup_winning_merge_byname_ID.iterrows():
    team_ids = row ["Team ID"]
    # print(team_ids)
    year_cl= row ["Year"]
    # print(year_cl)
    url = base_url.format(team_ids, year_cl)
    # print(url)
    response = requests.get(url)
    data = response.json()
    
    for player in data["teams"][0]["roster"]["roster"]:
        
        person = player["person"]["fullName"]
        # Create dictionary
        player_id = player["person"]["id"]
        # Printing for testing purposes
        #print("Name:", person)
        #print("ID:", player_id)
        player_list.append({
            "Name":person,
            "Player ID":player_id,
            "Team ID": team_ids,
            "Season":year_cl
        })
winning_df=pd.DataFrame(player_list)
winning_df.tail(30)
     

In [19]:
   
# Player Profile - Winning Team per Season
# Code by Ron Brennan

player_info_url = "https://statsapi.web.nhl.com/api/v1/people/{}"
player_info = []
player_stats = []

for index, row in winning_df.iterrows():
    player_nhl_id = row["Player ID"]
    winning_season = row["Season"]
    url = player_info_url.format(player_nhl_id)
    response = requests.get(url)
    data = response.json()

    for players in data["people"]:
            player_id = players["id"]
            person = players["fullName"]
            player_firstName = players["firstName"]
            player_lastName = players["lastName"]
            player_birthDate = players["birthDate"]
            player_birthCity = players["birthCity"]
            player_birthCountry = players["birthCountry"]
            try:
                player_birthStateProvince = players["birthStateProvince"]
            except Exception as e:
                player_birthStateProvince = 'N/A'     
            player_nationality = players["nationality"]
            player_height = players["height"]
            player_weight = players["weight"]
            try:
                player_shootsCatches = players["shootsCatches"]
            except Exception as e:
                player_shootsCatches = 'L'     
            player_primaryPositionCode = players["primaryPosition"]["code"]

            player_info.append({
                "player_id":player_id,
                "player_name":person,
                "player_firstName":player_firstName,
                "player_lastName":player_lastName,
                "player_birthDate":player_birthDate,
                "player_birthCity":player_birthCity,
                "player_birthCountry":player_birthCountry,
                "player_birthStateProvince": player_birthStateProvince,
                "player_nationality":player_nationality,
                "player_height":player_height,
                "player_weight":player_weight,
                "player_shootsCatches":player_shootsCatches,
                "player_primaryPositionCode":player_primaryPositionCode,
                "season": int(winning_season  )
            })
            
player_information_df = pd.DataFrame(player_info)
player_information_df = player_information_df.drop_duplicates(subset='player_id', keep="first")
player_information_sorted_df = player_information_df.sort_values(by=['player_id'])

goalie_df = player_information_sorted_df.loc[player_information_sorted_df['player_primaryPositionCode'] == 'G']

player_information_sorted_df.drop(['season'], axis=1, inplace = True)
player_information_sorted_df.to_csv("./schema/player_information_utf-8.csv", index=False, encoding = 'utf-8-sig') 
player_information_sorted_df.to_csv("./schema/player_information.csv", index=False) 
player_information_sorted_df.head()


,player_id,player_name,player_firstName,player_lastName,player_birthDate,player_birthCity,player_birthCountry,player_birthStateProvince,player_nationality,player_height,player_weight,player_shootsCatches,player_primaryPositionCode
107,8444919,Tommy Albelin,Tommy,Albelin,1964-05-21,Stockholm,SWE,N/A,SWE,"6' 2""",195,L,D
71,8444945,Glenn Anderson,Glenn,Anderson,1960-10-02,Vancouver,CAN,BC,CAN,"6' 1""",190,L,R
403,8445000,Dave Andreychuk,Dave,Andreychuk,1963-09-29,Hamilton,CAN,ON,CAN,"6' 4""",225,R,L
33,8445275,Tom Barrasso,Tom,Barrasso,1965-03-31,Boston,USA,MA,USA,"6' 3""",210,R,G
269,8445386,Ed Belfour,Ed,Belfour,1965-04-21,Carman,CAN,MB,CAN,"6' 0""",214,L,G


In [46]:
# Player Statistics - Winning Team (Skaters) per Season
# Code by Ron Brennan

player_stats_url = "https://statsapi.web.nhl.com/api/v1/people/{}/stats?stats=statsSingleSeason&season={}"
player_stats = []

winning_skaters_df = player_information_df.loc[player_information_df['player_primaryPositionCode'] != 'G']


for index, row in winning_skaters_df.iterrows():
     
        player_nhl_id = row["player_id"]
        season = row ["season"]
        player_birthyear_yyyy = row["player_birthDate"]
        win_season_yyyy = str(season)
        url = player_stats_url.format(player_nhl_id, season)
        response = requests.get(url)
        data_st = response.json()
        

        for player_st in data_st["stats"][0]["splits"]:
                player_season = player_st["season"]
                try:
                    player_timeOnIce = player_st["stat"]["timeOnIce"]
                except Exception as e:
                    player_timeOnIce = '-1'     
                player_assists = player_st["stat"]["assists"]
                player_goals = player_st["stat"]["goals"]
                player_pim = player_st["stat"]["pim"]
                player_shots= player_st["stat"]["shots"]
                player_games = player_st["stat"]["games"]
                try:
                    player_hits = player_st["stat"]["hits"]
                except Exception as e:
                    player_hits = '-1'
                player_powerPlayGoals = player_st["stat"]["powerPlayGoals"]
                player_powerPlayPoints = player_st["stat"]["powerPlayPoints"]
                try:
                    player_powerPlayTimeOnIce = player_st["stat"]["powerPlayTimeOnIce"]
                except Exception as e:
                    player_powerPlayTimeOnIce = '-1'
                try:
                    player_evenTimeOnIce = player_st["stat"]["evenTimeOnIce"]
                except Exception as e:
                    player_evenTimeOnIce = '-1'
                player_penaltyMinutes = player_st["stat"]["penaltyMinutes"]
                player_shotPct = player_st["stat"]["shotPct"]
                player_gameWinningGoals = player_st["stat"]["gameWinningGoals"]
                player_overTimeGoals = player_st["stat"]["overTimeGoals"]
                player_shortHandedGoals = player_st["stat"]["shortHandedGoals"]
                player_shortHandedPoints = player_st["stat"]["shortHandedPoints"]
                try:
                    player_blocked = player_st["stat"]["blocked"]
                except Exception as e:
                    player_blocked = '-1'
                player_plusMinus = player_st["stat"]["plusMinus"]
                player_points = player_st["stat"]["points"]
                try:
                    player_shifts = player_st["stat"]["shifts"]
                except Exception as e:
                    player_shifts = '-1'
                try:
                    player_timeOnIcePerGame = player_st["stat"]["timeOnIcePerGame"]
                except Exception as e:
                    player_timeOnIcePerGame = '-1'
                try:
                    player_evenTimeOnIcePerGame = player_st["stat"]["evenTimeOnIcePerGame"]
                except Exception as e:
                    player_evenTimeOnIcePerGame = '-1'
                try:
                    player_shortHandedTimeOnIcePerGame = player_st["stat"]["shortHandedTimeOnIcePerGame"]
                except Exception as e:
                    player_shortHandedTimeOnIcePerGame = '-1'
                try:
                    player_powerPlayTimeOnIcePerGame = player_st["stat"]["powerPlayTimeOnIcePerGame"]
                except Exception as e:
                    player_powerPlayTimeOnIcePerGame = '-1'

                player_season = win_season_yyyy[4:]
                player_birth_year = player_birthyear_yyyy[:4]
                winning_age = (int(player_season)) - (int(player_birth_year))
                
                player_stats.append({
                    "player_stats_id" : '', 
                    "player_id" : player_nhl_id, 
                    "season_id" : player_season,
                    "player_stats_age" : winning_age,
                    "player_stats_timeOnIce" : player_timeOnIce,
                    "player_stats_assists" : player_assists,
                    "player_stats_goals" : player_goals,
                    "player_stats_pim" : player_pim,
                    "player_stats_shots" : player_shots,
                    "player_stats_games" : player_games,
                    "player_stats_hits" : player_hits,
                    "player_stats_powerPlayGoals" : player_powerPlayGoals,
                    "player_stats_powerPlayPoints" : player_powerPlayPoints,
                    "player_stats_powerPlayTimeOnIce" : player_powerPlayTimeOnIce,
                    "player_stats_evenTimeOnIce" : player_evenTimeOnIce,
                    "player_stats_penaltyMinutes" : player_penaltyMinutes,
                    "player_stats_shotPct" : player_shotPct,
                    "player_stats_gameWinningGoals" : player_gameWinningGoals,
                    "player_stats_overTimeGoals" : player_overTimeGoals,
                    "player_stats_shortHandedGoals" : player_shortHandedGoals,
                    "player_stats_shortHandedPoints" : player_shortHandedPoints,
                    "player_stats_blocked" : player_blocked,
                    "player_stats_plusMinus" : player_plusMinus,
                    "player_stats_points" : player_points,
                    "player_stats_shifts" : player_shifts,
                    "player_stats_timeOnIcePerGame" : player_timeOnIcePerGame,
                    "player_stats_evenTimeOnIcePerGame" : player_evenTimeOnIcePerGame,
                    "player_stats_shortHandedTimeOnIcePerGame" : player_shortHandedTimeOnIcePerGame,
                    "player_stats_powerPlayTimeOnIcePerGame" : player_powerPlayTimeOnIcePerGame  
        })               
      
player_statistics = pd.DataFrame(player_stats)
player_statistics.to_csv("./schema/player_statistics_utf-8.csv", index=False, encoding = 'utf-8-sig')
player_statistics.to_csv("./schema/player_statistics.csv", index=False)
player_statistics.head()
      

,player_stats_id,player_id,season_id,player_stats_age,player_stats_timeOnIce,player_stats_assists,player_stats_goals,player_stats_pim,player_stats_shots,player_stats_games,...,player_stats_shortHandedGoals,player_stats_shortHandedPoints,player_stats_blocked,player_stats_plusMinus,player_stats_points,player_stats_shifts,player_stats_timeOnIcePerGame,player_stats_evenTimeOnIcePerGame,player_stats_shortHandedTimeOnIcePerGame,player_stats_powerPlayTimeOnIcePerGame
0,,8445629,1992,30,-1,16,10,58,51,58,...,1,3,-1,-6,26,-1,-1,-1,-1,-1
1,,8446013,1992,32,-1,0,0,175,16,50,...,0,0,-1,-6,0,-1,-1,-1,-1,-1
2,,8446050,1992,26,-1,4,0,111,26,43,...,0,0,-1,-12,4,-1,-1,-1,-1,-1
3,,8446117,1992,31,-1,58,11,87,232,64,...,0,2,-1,1,69,-1,-1,-1,-1,-1
4,,8446305,1992,24,-1,0,0,0,0,2,...,0,0,-1,0,0,-1,-1,-1,-1,-1


In [44]:
# Player Statistics - Winning Team (Goalies) per Season
# Code by Ron Brennan

player_stats_url = "https://statsapi.web.nhl.com/api/v1/people/{}/stats?stats=statsSingleSeason&season={}"
goalie_stats = []

for index, row in goalie_df.iterrows():
    player_nhl_id = row["player_id"]
    season = row ["season"]
    player_birthyear_yyyy = row["player_birthDate"]
    win_season_yyyy = str(season)
    url = player_stats_url.format(player_nhl_id, season)
    response = requests.get(url)
    data_st = response.json()

    for goalie_st in data_st["stats"][0]["splits"]:
           
            player_season = goalie_st["season"]
            goalie_timeOnIce = goalie_st["stat"]["timeOnIce"]
            try:
                goalie_ot = goalie_st["stat"]["ot"]
            except Exception as e:
                 goalie_ot = '-1'
            goalie_shutouts = goalie_st["stat"]["shutouts"]
            try:
                goalie_ties = goalie_st["stat"]["ties"]
            except Exception as e:
                 goalie_ties = '-1'         
            goalie_wins = goalie_st["stat"]["wins"]
            goalie_losses = goalie_st["stat"]["losses"]
            goalie_saves = goalie_st["stat"]["saves"]
            try:
                goalie_powerPlaySaves = goalie_st["stat"]["powerPlaySaves"]
            except Exception as e:
                goalie_powerPlaySaves = '-1'
            try:
                goalie_shortHandedSaves = goalie_st["stat"]["shortHandedSaves"]
            except Exception as e:
                goalie_shortHandedSaves = '-1'
            try:
                goalie_evenSaves = goalie_st["stat"]["evenSaves"]
            except Exception as e:
                goalie_evenSaves = '-1'
            try:
                goalie_shortHandedShots = goalie_st["stat"]["shortHandedShots"]
            except Exception as e:
                goalie_shortHandedShots = '-1'
            try:
                goalie_evenShots = goalie_st["stat"]["evenShots"]
            except Exception as e:
                goalie_evenShots = '-1'
            try:
                goalie_powerPlayShots = goalie_st["stat"]["powerPlayShots"]
            except Exception as e:
                goalie_powerPlayShots = '-1'
            try:
                goalie_savePercentage = goalie_st["stat"]["savePercentage"]
            except Exception as e:
                goalie_savePercentage = '-1'
            try:
                goalie_goalAgainstAverage = goalie_st["stat"]["goalAgainstAverage"]
            except Exception as e:
                goalie_goalAgainstAverage = '-1'
            goalie_games = goalie_st["stat"]["games"]
            goalie_gamesStarted = goalie_st["stat"]["gamesStarted"]
            goalie_shotsAgainst = goalie_st["stat"]["shotsAgainst"]
            goalie_goalsAgainst = goalie_st["stat"]["goalsAgainst"]
            try:
                goalie_timeOnIcePerGame = goalie_st["stat"]["timeOnIcePerGame"]
            except Exception as e:
                goalie_timeOnIcePerGame = '-1'          
            try:
                goalie_powerPlaySavePercentage = goalie_st["stat"]["powerPlaySavePercentage"]
            except Exception as e:
                goalie_powerPlaySavePercentage = '-1'
            try:
                goalie_shortHandedSavePercentage = goalie_st["stat"]["shortHandedSavePercentage"]
            except Exception as e:
                goalie_shortHandedSavePercentage = '-1'
            try:
                goalie_evenStrengthSavePercentage = goalie_st["stat"]["evenStrengthSavePercentage"]
            except Exception as e:
                goalie_evenStrengthSavePercentage = '-1'


            player_season = win_season_yyyy[4:]
            player_birth_year = player_birthyear_yyyy[:4]
            winning_age = (int(player_season)) - (int(player_birth_year))
            goalie_stats.append({
                "goalie_stats_id" : '',
                "player_id" : player_nhl_id, 
                "goalie_stats_season" : player_season,
                "goalie_stats_age" : winning_age,
                "goalie_stats_timeOnIce" : goalie_timeOnIce,
                "goalie_stats_goalie_ot" : goalie_ot,
                "goalie_stats_goalie_shutouts" : goalie_shutouts,
                "goalie_stats_goalie_ties" : goalie_ties,
                "goalie_stats_wins" : goalie_wins,
                "goalie_stats_goalie_losses" : goalie_losses,
                "goalie_stats_saves" : goalie_saves,               
                "goalie_stats_powerPlayGoals" : goalie_powerPlaySaves,
                "goalie_stats_powerPlayPoints" : goalie_shortHandedSaves,
                "goalie_stats_powerPlayTimeOnIce" : goalie_evenSaves,
                "goalie_stats_evenTimeOnIce" : goalie_shortHandedShots,
                "goalie_stats_penaltyMinutes" : goalie_evenShots,
                "goalie_stats_shotPct" : goalie_powerPlayShots,
                "goalie_stats_gameWinningGoals" : goalie_savePercentage,
                "goalie_stats_overTimeGoals" : goalie_goalAgainstAverage,
                "goalie_stats_shortHandedGoals" : goalie_games,
                "goalie_stats_primaryPositionCode" : goalie_gamesStarted,
                "goalie_stats_shortHandedPoints" : goalie_shotsAgainst,
                "goalie_stats_blocked" : goalie_goalsAgainst,
                "goalie_stats_plusMinus" : goalie_timeOnIcePerGame,
                "goalie_stats_points" : goalie_powerPlaySavePercentage,
                "goalie_stats_shortHandedSavePercentage" : goalie_powerPlaySavePercentage,
                "goalie_stats_timeOnIcePerGame" :goalie_shortHandedSavePercentage,
                "goalie_stats_evenTimeOnIcePerGame" : goalie_shortHandedSavePercentage,
                "goalie_stats_shortHandedTimeOnIcePerGame" : goalie_evenStrengthSavePercentage,

           })
           
goalie_statistics = pd.DataFrame(goalie_stats)
goalie_statistics.to_csv("./schema/goalie_statistics_utf-8.csv", index=False, encoding = 'utf-8-sig')
goalie_statistics.to_csv("./schema/goalie_statistics.csv", index=False)
goalie_statistics.head()
      

,goalie_stats_id,player_id,goalie_stats_season,goalie_stats_age,goalie_stats_timeOnIce,goalie_stats_goalie_ot,goalie_stats_goalie_shutouts,goalie_stats_goalie_ties,goalie_stats_wins,goalie_stats_goalie_losses,...,goalie_stats_shortHandedGoals,goalie_stats_primaryPositionCode,goalie_stats_shortHandedPoints,goalie_stats_blocked,goalie_stats_plusMinus,goalie_stats_points,goalie_stats_shortHandedSavePercentage,goalie_stats_timeOnIcePerGame,goalie_stats_evenTimeOnIcePerGame,goalie_stats_shortHandedTimeOnIcePerGame
0,,8445275,1992,27,3329:29,-1,1,9,25,22,...,57,57,1702,196,58:24,-1,-1,-1,-1,-1
1,,8445386,1999,34,3535:49,-1,5,9,35,15,...,61,59,1373,117,57:57,86.666667,86.666667,95.3125,95.3125,92.402659
2,,8446134,1993,25,40:00,-1,0,0,0,0,...,1,1,19,1,40:00,-1,-1,-1,-1,-1
3,,8446831,1996,26,2106:38,-1,1,7,22,6,...,37,35,1012,103,56:56,-1,-1,-1,-1,-1
4,,8447687,2002,37,3871:40,-1,5,8,41,15,...,65,64,1654,140,59:33,87.767584,87.767584,91.935484,91.935484,92.490119


Final Analysis/Conclusion:

- Based on our overall analysis and supporting data, there is no material difference regarding age groups and winning.  Years of experience requires additional analysis.
- There should be a heightened focus on recruiting European players.  Their hockey programs have significantly evolved, and the number of European players playing in the NHL is showing a steady increase, while the Canadian player representation is reducing.
- More PPG equates to a higher overall point value and not a substitute for regular goals.
- It is favorable and improves the likelihood of winning to have more consistency amongst contributing players vs having one or two superstars on the team.  The box plot comparing the +/- statistic supports this theor as the winning team has less outliers and more consistency when comparing the winning and losing team.  The +/- statistic is also the only key statistic analyzed that has a material difference(100% increase) in results when comparing the winning and losing team.
- Future analysis reviewing Stanley Cup Finals' data only, can provide greater variability in data points and provide additional trending, influencing additional recommendations.